In [1]:
%pip install numpy
%pip install pyvista --no-deps
%pip install matplotlib
%pip install scooby
%pip install webcolors

In [2]:
import re
import sys
import types
import requests
import ast

def extract_imports_from_ast(content):
    """
    Extracts VTK import statements from the given Python source content.

    Args:
        content (str): The Python source code as a string.

    Returns:
        list of tuple: A list of (module_name, class_name, alias) tuples.
    """
    results = []
    tree = ast.parse(content)

    for node in ast.walk(tree):
        # Handle standard imports: from vtkmodules.<module> import <class> as <alias>
        if isinstance(node, ast.ImportFrom) and node.module and node.module.startswith("vtkmodules."):
            module_name = node.module[len("vtkmodules."):]  # Strip the "vtkmodules." prefix
            for alias in node.names:
                class_name = alias.name
                as_name = alias.asname if alias.asname else alias.name
                results.append((module_name, class_name, as_name))

    return results

def create_mock_modules(parsed_imports):
    """
    Dynamically creates mock modules and injects them into sys.modules.

    Args:
        parsed_imports (list): A list of tuples (module_name, class_name, alias).
    """
    # Root vtkmodules package
    vtkmodules = types.ModuleType("vtkmodules")

    for module_name, class_name, alias in parsed_imports:
        # Split module_name into parts for nested modules
        parts = module_name.split(".")
        parent = vtkmodules

        # Create nested modules
        for part in parts:
            full_name = f"{parent.__name__}.{part}"
            if not hasattr(parent, part):
                submodule = types.ModuleType(full_name)
                setattr(parent, part, submodule)
                sys.modules[full_name] = submodule
            parent = getattr(parent, part)

        # Create mock class
        mock_class = type(class_name, (), {})
        setattr(parent, alias, mock_class)

    # Inject the root vtkmodules into sys.modules
    sys.modules["vtkmodules"] = vtkmodules

# URL of the source file
url = "https://raw.githubusercontent.com/pyvista/pyvista/refs/heads/main/pyvista/core/_vtk_core.py"

# Fetch the content from the URL
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch content from URL: {url}")

content = response.text

# Parse imports
parsed_imports = extract_imports_from_ast(content)

# Create mock modules
create_mock_modules(parsed_imports)

# Test the structure
# print("Mocked modules and classes:")
# for module_name, class_name, alias in parsed_imports:
#     print(f"Module: {module_name}, Class: {class_name}, Alias: {alias}")

In [3]:
# Expected to fail
import pyvista

/lib/python3.12/site-packages/pyvista/core/_vtk_core.py:455: UserWarning: Unable to detect VTK version. Defaulting to v4.0.0
  warnings.warn("Unable to detect VTK version. Defaulting to v4.0.0")


<class 'TypeError'>: int() argument must be a string, a bytes-like object or a real number, not 'type'

In [4]:
import re
import requests

def parse_vtk_constants(content):
    """
    Parses VTK constants with numeric values from the given content.

    Args:
        content (str): The source content to parse.

    Returns:
        dict: A dictionary of constants and their corresponding values.
    """
    # Regex to match lines like "VTK_* = <number>,"
    pattern = re.compile(r"^\s*VTK_(\w+):\s*(\d+),")
    constants = {}

    for line in content.splitlines():
        match = pattern.match(line)
        if match:
            name = f"VTK_{match.group(1)}"
            value = int(match.group(2))
            constants[name] = value

    return constants

# URL of the source file
url = "https://raw.githubusercontent.com/Kitware/vtk-js/refs/heads/master/Sources/Common/DataModel/CellTypes/Constants.js"

# Fetch the content from the URL
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch content from URL: {url}")

content = response.text

# Parse constants using the regex
vtk_constants = parse_vtk_constants(content)

# Display the constants
print("Parsed VTK Constants:")
for name, value in vtk_constants.items():
    print(f"{name} = {value}")

# Generate mock definitions
# print("\nGenerated Mock Definitions:")
# for name, value in vtk_constants.items():
#     print(f"{name} = {value}")

# Dynamically extend the previously built mock
# if "pyvista.core._vtk_core" not in sys.modules:
#     raise ImportError("_vtk_core mock must already exist")

# vtk_core_mock = sys.modules["pyvista.core._vtk_core"]

# # Adding parsed constants from the VTK CellType specification

# for name, value in vtk_constants.items():
#     setattr(vtk_core_mock, name, value)

if "vtkmodules" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtkmodules = sys.modules["vtkmodules"]

# Adding parsed constants from the VTK CellType specification

for name, value in vtk_constants.items():
    setattr(vtkmodules, name, value)

Parsed VTK Constants:
VTK_EMPTY_CELL = 0
VTK_VERTEX = 1
VTK_POLY_VERTEX = 2
VTK_LINE = 3
VTK_POLY_LINE = 4
VTK_TRIANGLE = 5
VTK_TRIANGLE_STRIP = 6
VTK_POLYGON = 7
VTK_PIXEL = 8
VTK_QUAD = 9
VTK_TETRA = 10
VTK_VOXEL = 11
VTK_HEXAHEDRON = 12
VTK_WEDGE = 13
VTK_PYRAMID = 14
VTK_PENTAGONAL_PRISM = 15
VTK_HEXAGONAL_PRISM = 16
VTK_QUADRATIC_EDGE = 21
VTK_QUADRATIC_TRIANGLE = 22
VTK_QUADRATIC_QUAD = 23
VTK_QUADRATIC_POLYGON = 36
VTK_QUADRATIC_TETRA = 24
VTK_QUADRATIC_HEXAHEDRON = 25
VTK_QUADRATIC_WEDGE = 26
VTK_QUADRATIC_PYRAMID = 27
VTK_BIQUADRATIC_QUAD = 28
VTK_TRIQUADRATIC_HEXAHEDRON = 29
VTK_QUADRATIC_LINEAR_QUAD = 30
VTK_QUADRATIC_LINEAR_WEDGE = 31
VTK_BIQUADRATIC_QUADRATIC_WEDGE = 32
VTK_BIQUADRATIC_QUADRATIC_HEXAHEDRON = 33
VTK_BIQUADRATIC_TRIANGLE = 34
VTK_CUBIC_LINE = 35
VTK_CONVEX_POINT_SET = 41
VTK_POLYHEDRON = 42
VTK_PARAMETRIC_CURVE = 51
VTK_PARAMETRIC_SURFACE = 52
VTK_PARAMETRIC_TRI_SURFACE = 53
VTK_PARAMETRIC_QUAD_SURFACE = 54
VTK_PARAMETRIC_TETRA_REGION = 55
VTK_PARAMETRIC_HEX

In [5]:
# import re
# import requests

# def parse_celltype_constants(content):
#     """
#     Parses the VTK constants (e.g., VTK_EMPTY_CELL) from celltype.py content.

#     Args:
#         content (str): The Python source code to parse.

#     Returns:
#         set: A set of VTK constants (e.g., VTK_EMPTY_CELL, VTK_VERTEX).
#     """
#     # Regex to match `_vtk.VTK_*` references
#     pattern = re.compile(r"value=_vtk\.(VTK_\w+)")
#     constants = set()

#     for line in content.splitlines():
#         match = pattern.search(line)
#         if match:
#             constants.add(match.group(1))

#     return constants

# def check_missing_constants(parsed_constants, mock_constants):
#     """
#     Compares parsed constants against the mock constants and identifies any missing ones.

#     Args:
#         parsed_constants (set): A set of parsed constants from the source file.
#         mock_constants (set): A set of mock constants defined in the mock module.

#     Returns:
#         set: A set of constants missing from the mock module.
#     """
#     return parsed_constants - mock_constants

# # URL of the source file
# url = "https://raw.githubusercontent.com/pyvista/pyvista/refs/heads/main/pyvista/core/celltype.py"

# # Fetch the content from the URL
# response = requests.get(url)
# if response.status_code != 200:
#     raise Exception(f"Failed to fetch content from URL: {url}")

# content = response.text

# # Parse constants from the source file
# parsed_constants = parse_celltype_constants(content)

# # Compare and find missing constants
# missing_constants = check_missing_constants(parsed_constants, vtk_constants)

# # Display results
# print("Parsed Constants:")
# print(parsed_constants)
# print("\nMock Constants:")
# print(mock_constants)
# print("\nMissing Constants:")
# print(missing_constants)

In [6]:
import re
import requests
import sys

# Function to parse constants from the VTK JavaScript file
def parse_vtk_constants(content):
    """
    Parses VTK constants with numeric values from the given content.

    Args:
        content (str): The source content to parse.

    Returns:
        dict: A dictionary of constants and their corresponding values.
    """
    # Regex to match lines like "VTK_*: <number>,"
    pattern = re.compile(r"^\s*VTK_(\w+):\s*(\d+),")
    constants = {}

    for line in content.splitlines():
        match = pattern.match(line)
        if match:
            name = f"VTK_{match.group(1)}"
            value = int(match.group(2))
            constants[name] = value

    return constants

# Function to parse constants from the PyVista `celltype.py` file
def parse_celltype_constants(content):
    """
    Parses the VTK constants (e.g., VTK_EMPTY_CELL) from celltype.py content.

    Args:
        content (str): The Python source code to parse.

    Returns:
        set: A set of VTK constants (e.g., VTK_EMPTY_CELL, VTK_VERTEX).
    """
    # Regex to match `_vtk.VTK_*` references
    pattern = re.compile(r"value=_vtk\.(VTK_\w+)")
    constants = set()

    for line in content.splitlines():
        match = pattern.search(line)
        if match:
            constants.add(match.group(1))

    return constants

# Function to compare parsed constants with mock constants
def check_missing_constants(parsed_constants, mock_constants):
    """
    Compares parsed constants against the mock constants and identifies any missing ones.

    Args:
        parsed_constants (set): A set of parsed constants from the source file.
        mock_constants (set): A set of mock constants defined in the mock module.

    Returns:
        set: A set of constants missing from the mock module.
    """
    return parsed_constants - mock_constants

# URL of the VTK constants (JavaScript)
vtk_constants_url = "https://raw.githubusercontent.com/Kitware/vtk-js/refs/heads/master/Sources/Common/DataModel/CellTypes/Constants.js"

# Fetch and parse VTK constants
vtk_response = requests.get(vtk_constants_url)
if vtk_response.status_code != 200:
    raise Exception(f"Failed to fetch VTK constants from URL: {vtk_constants_url}")

vtk_constants_content = vtk_response.text
vtk_constants = parse_vtk_constants(vtk_constants_content)

beizer_constants = {
    "VTK_BEZIER_CURVE": 75,
    "VTK_BEZIER_TRIANGLE": 76,
    "VTK_BEZIER_QUADRILATERAL": 77,
    "VTK_BEZIER_TETRAHEDRON": 78,
    "VTK_BEZIER_HEXAHEDRON": 79,
    "VTK_BEZIER_WEDGE": 80,
    "VTK_BEZIER_PYRAMID": 10000,
    "VTK_TRIQUADRATIC_PYRAMID": 10001,
}

vtk_constants.update(beizer_constants)


# Dynamically extend the previously built mock
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core_mock = sys.modules["pyvista.core._vtk_core"]
for name, value in vtk_constants.items():
    setattr(vtk_core_mock, name, value)

# URL of the PyVista `celltype.py`
celltype_url = "https://raw.githubusercontent.com/pyvista/pyvista/refs/heads/main/pyvista/core/celltype.py"

# Fetch and parse celltype constants
celltype_response = requests.get(celltype_url)
if celltype_response.status_code != 200:
    raise Exception(f"Failed to fetch CellType constants from URL: {celltype_url}")

celltype_content = celltype_response.text
celltype_constants = parse_celltype_constants(celltype_content)

# Check for missing constants
# missing_constants = check_missing_constants(celltype_constants, set(vtk_constants.keys()))

# # Display results
# print("Parsed VTK Constants:")
# print(vtk_constants)

# print("\nParsed CellType Constants:")
# print(celltype_constants)

# print("\nMissing Constants:")
# print(missing_constants)

In [7]:
# Ensure the mock module is already initialized
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core = sys.modules["pyvista.core._vtk_core"]

# Mock the necessary classes for VTK Rendering
class vtkMathTextFreeTypeTextRenderer:
    pass  # Placeholder class

class vtkVectorText:
    pass  # Placeholder class

class vtkLabelPlacementMapper:
    pass  # Placeholder class

# Adding these classes to the mock module
vtk_core.vtkMathTextFreeTypeTextRenderer = vtkMathTextFreeTypeTextRenderer
vtk_core.vtkVectorText = vtkVectorText
vtk_core.vtkLabelPlacementMapper = vtkLabelPlacementMapper

# Verify the classes were added successfully
print(f"vtkMathTextFreeTypeTextRenderer: {vtk_core.vtkMathTextFreeTypeTextRenderer}")
print(f"vtkVectorText: {vtk_core.vtkVectorText}")
print(f"vtkLabelPlacementMapper: {vtk_core.vtkLabelPlacementMapper}")

vtkMathTextFreeTypeTextRenderer: <class '__main__.vtkMathTextFreeTypeTextRenderer'>
vtkVectorText: <class '__main__.vtkVectorText'>
vtkLabelPlacementMapper: <class '__main__.vtkLabelPlacementMapper'>


In [8]:
# Define vtkDataObject constants
class vtkDataObject:
    FIELD_ASSOCIATION_POINTS = 0
    FIELD_ASSOCIATION_CELLS = 1
    FIELD_ASSOCIATION_NONE = 2
    FIELD_ASSOCIATION_ROWS = 3

# Add vtkDataObject to the mock module
vtk_core.vtkDataObject = vtkDataObject

In [9]:
import numpy as np
import sys

# Ensure the mock module is already initialized
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core_mock = sys.modules["pyvista.core._vtk_core"]

# Mock VTKArray class that mimics a numpy array-like behavior
class VTKArray(np.ndarray):
    def __new__(cls, shape, dtype=float, buffer=None, offset=0, strides=None, order=None):
        # Create the ndarray instance
        obj = np.ndarray.__new__(cls, shape, dtype, buffer, offset, strides, order)
        return obj
    
    def __array_finalize__(self, obj):
        """
        This method is called when an array is modified or used in an operation.
        We override it to ensure the mock array behaves correctly with NumPy.
        """
        if obj is None:  # In case of initialization
            return
        # Ensure no undefined attributes are accessed
        if not hasattr(self, '_vtk_array_initialized'):
            self._vtk_array_initialized = True

    def __getattr__(self, name):
        # Handle missing attributes by returning a default value or custom behavior
        print(f"Mocking __getattr__ for missing attribute: {name}")
        # Example behavior: return a custom value or raise an error
        return f"Mocked value for {name}"

# Add the mock VTKArray to the mock module
vtk_core_mock.VTKArray = VTKArray

# Test the VTKArray class and the __array_finalize__ method
vtk_array = vtk_core_mock.VTKArray((3, 3), dtype=float)
print(f"VTKArray created with shape {vtk_array.shape}")

# Perform an operation to trigger __array_finalize__
vtk_array[0] = 10
print(f"VTKArray after modification: {vtk_array}")

# Check if the internal flag set in __array_finalize__ is working
print(f"VTKArray internal flag: {getattr(vtk_array, '_vtk_array_initialized', False)}")

# Now let's try accessing an undefined attribute to trigger __getattr__
print(vtk_array.undefined_attribute)  # This will trigger __getattr__

VTKArray created with shape (3, 3)
Mocking __getattr__ for missing attribute: _vtk_array_initialized
VTKArray after modification: [[1.00000000e+001 1.00000000e+001 1.00000000e+001]
 [3.25632936e-316 9.88131292e-324 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000]]
Mocking __getattr__ for missing attribute: _vtk_array_initialized
VTKArray internal flag: Mocked value for _vtk_array_initialized
Mocking __getattr__ for missing attribute: undefined_attribute
Mocked value for undefined_attribute


In [10]:
import sys

# Ensure the mock module is already initialized
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core = sys.modules["pyvista.core._vtk_core"]

# Mock vtkWeakReference class
class vtkWeakReference:
    def __init__(self):
        self._reference = None

    def Set(self, obj):
        # Store the reference to the actual object
        self._reference = obj
    
    def Get(self):
        # Return the actual referenced object
        return self._reference

# Mock vtkPolyData class
class vtkPolyData:
    def __init__(self):
        self.data = "Sample VTK data"

    def Set(self, data):
        self.data = data  # Simulate setting some data in the object

# Add mocked classes to the mock module
vtk_core.vtkWeakReference = vtkWeakReference
vtk_core.vtkPolyData = vtkPolyData

# Test using vtkWeakReference to hold a reference to vtkPolyData
weak_ref = vtk_core.vtkWeakReference()
vtk_object = vtk_core.vtkPolyData()

# Set the reference (store the vtkPolyData object in the vtkWeakReference)
weak_ref.Set(vtk_object)

# Get the reference and call 'Set' on the actual object (dereferencing the weak reference)
resolved_object = weak_ref.Get()
resolved_object.Set("Updated VTK data")

# Test that the Set method worked on the referenced object
print(f"Resolved object data: {resolved_object.data}")


Resolved object data: Updated VTK data


In [11]:
import sys
import types

# Ensure the mock module is already initialized
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core_mock = sys.modules["pyvista.core._vtk_core"]

# Create mock modules for vtkRenderingFreeType and vtkRenderingLabel
vtkRenderingFreeType_mock = types.ModuleType("vtkmodules.vtkRenderingFreeType")
vtkRenderingLabel_mock = types.ModuleType("vtkmodules.vtkRenderingLabel")

# Mock classes for vtkRenderingFreeType
class vtkMathTextFreeTypeTextRenderer:
    pass  # Placeholder class

class vtkVectorText:
    pass  # Placeholder class

# Add the mocked classes to the vtkRenderingFreeType mock module
vtkRenderingFreeType_mock.vtkMathTextFreeTypeTextRenderer = vtkMathTextFreeTypeTextRenderer
vtkRenderingFreeType_mock.vtkVectorText = vtkVectorText

# Mock class for vtkRenderingLabel
class vtkLabelPlacementMapper:
    pass  # Placeholder class

# Add the mocked class to the vtkRenderingLabel mock module
vtkRenderingLabel_mock.vtkLabelPlacementMapper = vtkLabelPlacementMapper

# Add the mock modules to the mock core module (vtk_core_mock)
vtk_core_mock.vtkRenderingFreeType = vtkRenderingFreeType_mock
vtk_core_mock.vtkRenderingLabel = vtkRenderingLabel_mock

# Add the mock modules to sys.modules to simulate actual imports
sys.modules["vtkmodules.vtkRenderingFreeType"] = vtkRenderingFreeType_mock
sys.modules["vtkmodules.vtkRenderingLabel"] = vtkRenderingLabel_mock

# Now, we can test the imports
from pyvista.core._vtk_core import vtkRenderingFreeType, vtkRenderingLabel

print(f"vtkRenderingFreeType.vtkMathTextFreeTypeTextRenderer: {vtkRenderingFreeType.vtkMathTextFreeTypeTextRenderer}")
print(f"vtkRenderingFreeType.vtkVectorText: {vtkRenderingFreeType.vtkVectorText}")
print(f"vtkRenderingLabel.vtkLabelPlacementMapper: {vtkRenderingLabel.vtkLabelPlacementMapper}")

vtkRenderingFreeType.vtkMathTextFreeTypeTextRenderer: <class '__main__.vtkMathTextFreeTypeTextRenderer'>
vtkRenderingFreeType.vtkVectorText: <class '__main__.vtkVectorText'>
vtkRenderingLabel.vtkLabelPlacementMapper: <class '__main__.vtkLabelPlacementMapper'>


In [12]:
import sys
import types

# Ensure the mock module is already initialized
if "pyvista.core._vtk_core" not in sys.modules:
    raise ImportError("_vtk_core mock must already exist")

vtk_core = sys.modules["pyvista.core._vtk_core"]

# Create the vtkCommonExecutionModel mock module
vtkcommonexecutionmodel = types.ModuleType("vtkmodules.vtkCommonExecutionModel")

# Mock vtkAlgorithm class with SINGLE_PRECISION as a class attribute
class vtkAlgorithm:
    SINGLE_PRECISION = 0  # Class-level attribute
    
    def SetInputData(self, data):
        pass
    
    def GetOutput(self):
        return "mock_output"

# Add vtkAlgorithm class directly to vtkmodules.vtkCommonExecutionModel in sys.modules
vtkcommonexecutionmodel.vtkAlgorithm = vtkAlgorithm

# Add vtkmodules.vtkCommonExecutionModel to the vtkmodules mock
# vtkmodules.vtkCommonExecutionModel = vtkcommonexecutionmodel

# Add vtkmodules to sys.modules to simulate actual imports
sys.modules["vtkmodules.vtkCommonExecutionModel"] = vtkcommonexecutionmodel

sys.modules["pyvista.core._vtk_core"] = vtk_core

# Now we can test the import of vtkAlgorithm from the correct mock module
from vtkmodules.vtkCommonExecutionModel import vtkAlgorithm


# Access the mocked SINGLE_PRECISION constant
print(f"vtkAlgorithm.SINGLE_PRECISION: {vtkAlgorithm.SINGLE_PRECISION}")

# Print the structure of vtkAlgorithm after import

vtkAlgorithm.SINGLE_PRECISION: 0


In [13]:
from vtkmodules.vtkCommonExecutionModel import vtkAlgorithm

In [14]:
from pyvista.core._vtk_core import vtkAlgorithm as va

In [15]:
# TODO: Very hacky for now. Import issues..
va.SINGLE_PRECISION = 0
va.DOUBLE_PRECISION = 0

In [16]:
from pyvista._plot import plot

In [17]:
import importlib
importlib.invalidate_caches()

In [18]:
del sys.modules['pyvista._plot']
import pyvista

Mocking __getattr__ for missing attribute: _vtk_array_initialized


<class 'AttributeError'>: 'vtkIdTypeArray' object has no attribute 'GetDataTypeSize'

In [19]:
import sys
import types

# Step 1: Create a mock for vtkIdTypeArray
vtkmodules = types.ModuleType("vtkmodules")
sys.modules["vtkmodules"] = vtkmodules

vtkCommonDataModel = types.ModuleType("vtkmodules.vtkCommonDataModel")
sys.modules["vtkmodules.vtkCommonDataModel"] = vtkCommonDataModel

# Step 2: Create the mock vtkIdTypeArray class
class vtkIdTypeArray:
    def __init__(self):
        # Example attribute
        self.data_type_size = 4  # Mock value for data type size

    def GetDataTypeSize(self):
        # Return a mocked data type size
        return self.data_type_size

# Step 3: Add MockvtkIdTypeArray to the mock vtkCommonDataModel module
vtkCommonDataModel.vtkIdTypeArray = vtkIdTypeArray

vtk_core = sys.modules["pyvista.core._vtk_core"]
vtk_core.vtkIdTypeArray = vtkIdTypeArray

In [20]:
class vtkVersion:
    major = 9
    @staticmethod
    def GetVTKMajorVersion():
        return 9

    @staticmethod
    def GetVTKMinorVersion():
        return 4

    @staticmethod
    def GetVTKBuildVersion():
        return 1

vtkCommonCore = types.ModuleType("vtkmodules.vtkCommonCore")
vtkCommonCore.vtkVersion = vtkVersion
vtkmodules.vtkCommonCore = vtkCommonCore
vtk_core.vtk_version_info = vtkVersion

In [21]:
import sys
import types

# Step 1: Create a mock for vtkOutputWindow and vtkStringOutputWindow
vtkmodules_mock = types.ModuleType("vtkmodules")
sys.modules["vtkmodules"] = vtkmodules_mock

vtkCommonCore_mock = types.ModuleType("vtkmodules.vtkCommonCore")
sys.modules["vtkmodules.vtkCommonCore"] = vtkCommonCore_mock

# Step 2: Create vtkOutputWindow with SetInstance
class vtkOutputWindow:
    def __init__(self):
        self.instance_set = False
    
    def SetInstance(self, instance):
        # Mock behavior of SetInstance
        print(f"SetInstance called with: {instance}")
        self.instance_set = True

# Step 3: Create vtkStringOutputWindow with AddObserver
class vtkStringOutputWindow:
    def __init__(self):
        self.observers = []
    
    def AddObserver(self, event, callback):
        # Mock behavior of AddObserver
        print(f"AddObserver called with event: {event} and callback: {callback}")
        self.observers.append((event, callback))

# Step 4: Add vtkOutputWindow and vtkStringOutputWindow to the mock vtkCommonCore module
vtkCommonCore_mock.vtkOutputWindow = vtkOutputWindow
vtkCommonCore_mock.vtkStringOutputWindow = vtkStringOutputWindow

vtk_core.vtkOutputWindow = vtkOutputWindow
vtk_core.vtkStringOutputWindow = vtkStringOutputWindow

In [22]:
import pyvista

SetInstance called with: <__main__.vtkStringOutputWindow object at 0x5284120>
AddObserver called with event: ErrorEvent and callback: <pyvista.core.utilities.observers.Observer object at 0x4e5e250>


## IT'S ALIVE!

In [23]:
import json
import webcolors
from IPython.display import display, HTML

In [24]:
class myCylinderSource():
    def __init__(
        self,
        center  = (0.0, 0.0, 0.0),
        direction = (1.0, 0.0, 0.0),
        radius = 0.5,
        height = 1.0,
        capping = True,
        resolution = 100,
    ) -> None:
        self.center = center
        self.direction = direction
        self.radius = radius
        self.height = height
        self.resolution = resolution
        self.capping = capping
    def SetHeight(self, x):
        self.height = x
    def SetRadius(self, x):
        self.radius = x
    def SetResolution(self, x):
        self.resolution = x

In [25]:
class myPlotter():
    template = """
<div id="vtk-container" style="width: WIDTHpx; height: HEIGHTpx; border: 1px solid black;"></div>
<script type="text/javascript">
  if (typeof vtk === "undefined") {
    var script = document.createElement("script");
    script.src = "https://unpkg.com/vtk.js";
    document.head.appendChild(script);
    script.onload = function() {
      console.log("vtk.js loaded successfully");
      // Your vtk.js initialization code here
    };
  } else {
    console.log("vtk.js is already loaded");
    // Your vtk.js initialization code here
    var fullScreenRenderer = vtk.Rendering.Misc.vtkFullScreenRenderWindow.newInstance({
      rootContainer: document.getElementById('vtk-container'),
    });

    MESH_TEMPLATE

    renderer.resetCamera();
    var renderWindow = fullScreenRenderer.getRenderWindow();
    renderWindow.render();
  }
</script>
"""
    mesh_template = """
    var object_params_ID = OBJECT_PARAMETERS
    var actor_params_ID = ACTOR_PARAMETERS
    
    var actor_ID = vtk.Rendering.Core.vtkActor.newInstance();
    actor_ID.getProperty().setColor(...actor_params_ID.color);
    actor_ID.getProperty().setEdgeVisibility(actor_params_ID.show_edges);
    
    var mapper_ID = vtk.Rendering.Core.vtkMapper.newInstance();
    var cone_ID = vtk.Filters.Sources.vtkConeSource.newInstance();

    var obj_ID = vtk.Filters.Sources.vtkCylinderSource.newInstance(object_params_ID);

    actor_ID.setMapper(mapper_ID);
    mapper_ID.setInputConnection(obj_ID.getOutputPort());

    var renderer = fullScreenRenderer.getRenderer();
    renderer.addActor(actor_ID);
"""
    def __init__(self, height=400, width=600):
        self.height = height
        self.width = width
        self.meshes = []
        self.next_mesh_id = 1
    def add_mesh(self, mesh, color="black", show_edges=False):
        # Typing the word 'show' here seems to flip the actual realtime display. I don't understand that..
        # It does suggest we could have some realtime updating though (i.e. render continously on cell change
        # Or 'render if it compiles' on cell change
        self.meshes.append([self.next_mesh_id, mesh, color, show_edges])
        self.next_mesh_id = self.next_mesh_id + 1
    def show(self, debug=False):
        rendered_meshes = ""
        for [mesh_id, mesh, color, show_edges] in self.meshes:
            object_params = json.dumps(
                {
                    'height': mesh.height,
                    'radius': mesh.radius,
                    'resolution': mesh.resolution,
                }
            )
            # NOTE: We could move the RGB bits into JS and parse with canvas
            actor_params = json.dumps(
                {
                    'color':list(webcolors.name_to_rgb(color)),
                    'show_edges': show_edges,
                }
            )
            rendered_mesh = self.mesh_template \
                .replace("OBJECT_PARAMETERS", object_params) \
                .replace("ACTOR_PARAMETERS", actor_params) \
                .replace("ID", str(mesh_id))
            rendered_meshes += rendered_mesh
        # print(rendered_mesh)
        output = self.template \
            .replace("HEIGHT", str(self.height)) \
            .replace("WIDTH", str(self.width)) \
            .replace("MESH_TEMPLATE", rendered_meshes)
        if debug:
            print(output)
        display(HTML(output))

In [26]:
pyvista.CylinderSource = myCylinderSource

In [27]:
pyvista.Plotter = myPlotter

In [28]:
cylinder = pyvista.CylinderSource()
cylinder.SetHeight(2)
cylinder.SetRadius(2)

In [29]:
plotter = pyvista.Plotter()
plotter.add_mesh(cylinder, color="yellow", show_edges=True)
plotter.show()